In [3]:
import pandas as pd
import pandas_gbq as pgbq
from scipy import stats

google_project_id = 'solid-ridge-104914'

In [5]:
# If already calculated, load message_net

message_net = pd.read_csv('~/community_health/data/message_network_properties.csv')

In [8]:
# Or, calculate network properties 
# WARNING: This query takes over 30 mins! No harm possible, just annoying.

with open('../sql/cumulative_networks/message_net.txt') as query_file:
    message_query = query_file.read()

message_net = pgbq.read_gbq(message_query, google_project_id, dialect='standard')
pgbq.to_gbq(message_net, 'community_networks.transaction_net', google_project_id, if_exists='replace')

ServerNotFoundError: Unable to find the server at www.googleapis.com

In [6]:
message_net

edge_type   cid                                               name  \
0     transaction     1               Vaudreuil-Dorion & Surrounding Areas   
1     transaction     2                           Turks and Caicos Islands   
2     transaction     3                            West Island of Montreal   
3     transaction     4                                              Laval   
4     transaction     5                            Valleyfield et environs   
5     transaction     6                         Casa Grande Buy-Sell-Trade   
6     transaction     7          Whidbey Island Neighbors' Classifieds, WA   
7     transaction     8  Mount Forest, Arthur & Fergus Ontario Buy and ...   
8     transaction    11                         Fort Lewis Flea Market, WA   
9     transaction    12                New Richmond Wisconsin Buy and Sell   
10    transaction    13                      Mohawk Valley, NY Garage Sale   
11    transaction    14          Orangeville/Shelburne, Ontario Buy & Sell   
12    transaction    15                   Swan River Valley, MB Buy & Sell   
13    transaction    16                          Parkland, MB Bidding Wars   
14    transaction    18                      Moore County, NC Buy and Sell   
15    transaction    19                               Minnesota East Metro   
16    transaction    20                          Opelika-Auburn Online, AL   
17    transaction    21             Weatherford, TX and surrounding cities   
18    transaction    22                              PUTCO Tennessee Group   
19    transaction    23                                               Test   
20    transaction    24                             Norterra, AZ Swip Swap   
21    transaction    25                             Easton, MD Swap & Sell   
22    transaction    26                            Shawville, QC Yard Sale   
23    transaction    27                     Geneseo, IL Area Hometown Swap   
24    transaction    28                             Upstate South Carolina   
25    transaction    29  Bishops Stortford Hertfordshire (UK) Free Buyi...   
26    transaction    30                           Port Huron Area Swap, MI   
27    transaction    31                               Anthem Treasures, AZ   
28    transaction    32                        Lynchburg Area Yardsale, VA   
29    transaction    33                          Quad Cities Iowa/Illinois   
...           ...   ...                                                ...   
1986  transaction  2701                              King, ON Buy and Sell   
1987  transaction  2702                              Essa, ON Buy and Sell   
1988  transaction  2703                       Oro-Medonte, ON Buy and Sell   
1989  transaction  2704                       Springwater, ON Buy and Sell   
1990  transaction  2705                            Selwyn, ON Buy and Sell   
1991  transaction  2706                         Clearview, ON Buy and Sell   
1992  transaction  2707                         St. Clair, ON Buy and Sell   
1993  transaction  2708                     Thames Centre, ON Buy and Sell   
1994  transaction  2711                         Wellesley, ON Buy and Sell   
1995  transaction  2712                          Mapleton, ON Buy and Sell   
1996  transaction  2713                   Georgian Bluffs, ON Buy and Sell   
1997  transaction  2714                    North Dumfries, ON Buy and Sell   
1998  transaction  2716                   Penetanguishene, ON Buy and Sell   
1999  transaction  2717                    Cavan Monaghan, ON Buy and Sell   
2000  transaction  2718                         Champlain, ON Buy and Sell   
2001  transaction  2719                             Minto, ON Buy and Sell   
2002  transaction  2720                              Mono, ON Buy and Sell   
2003  transaction  2725                     Huron-Kinloss, ON Buy and Sell   
2004  transaction  2726           Otonabee-South Monaghan, ON Buy and Sell   
2005  transaction  2727       

In [7]:
# Cross correlate community facts and output key correlations

variables = list(message_net)[4:len(message_net)] #this is not a good way to create the list of variables

message_corr_matrix = pd.DataFrame(index=variables, columns=variables)
message_key_correlations = pd.DataFrame(columns=['pair','coefficient'])
corr_checked = list()

i = 0
j = 0

for i in range(len(variables)):
    for j in range(len(variables)):
        pair = variables[i]+','+variables[j]
        rho, pval = stats.spearmanr(message_net[[variables[i],variables[j]]])
        message_corr_matrix.set_value(variables[i], variables[j], rho)
        if pair not in corr_checked and rho < 0.99 and (rho > 0.5 or rho < -0.5):
            message_key_correlations.loc[len(message_key_correlations)] = [pair, rho]
            corr_checked.append(variables[j]+','+variables[i])

message_key_correlations.sort_values(by='coefficient', ascending=False, inplace=True)
message_key_correlations.reset_index(drop=True, inplace=True)

#Save to BigQuery
#pgbq.to_gbq(message_corr_matrix, 'community_networks.message_corr_matrix', google_project_id, if_exists='replace')
#pgbq.to_gbq(message_key_correlations, 'community_networks.message_key_correlations', google_project_id, if_exists='replace')
message_key_correlations

pair  coefficient
0                  nodes,avg_weight     0.805421
1   actual_edges,total_transactions     0.803515
2             avg_degree,avg_weight    -0.827630
3  network_density_X_100,avg_weight    -0.832252
4                  nodes,avg_degree    -0.952044
5       nodes,network_density_X_100    -0.979852

Transaction-Message observations:

 - node count and avg edge weight are NOT correlated in transactions, but ARE strongly correlated in messages
         ie, the bigger the community, the more messages between pairs. Almost certainly confounded by time.
 - node count and avg degree are NOT correlated in transactions, but ARE strongly inversely correlated in messages
         ie, the bigger the community, the fewer members one has messaged. Likely confounded by inactive users, maybe by ease of shopping due to more selection.
       